# Data Cleaning

---
Notebook ini adalah program untuk cleaning scraping data, clean data akan digunakan untuk proses analisis dan pemodelan
---

# Import Libaries

In [131]:
# Import Libraries
import json
import pandas as pd
import numpy as np
import re
import ast
from langdetect import detect, DetectorFactory
import warnings
warnings.filterwarnings('ignore')


# Load Data

menggabungkan semua json menjadi satu dataframe

In [132]:
# Read file JSON
with (open('./scrapping_data/adidas.json') as f1,
      open('./scrapping_data/nb.json') as f2,
      open('./scrapping_data/nike.json') as f3,
      open('./scrapping_data/puma.json') as f4,
      open('./scrapping_data/reebok.json') as f5):
    data1 = json.load(f1)
    data2 = json.load(f2)
    data3 = json.load(f3)
    data4 = json.load(f4)
    data5 = json.load(f5)

# Combine
combine = data1 + data2 + data3 + data4 + data5
# Save as backup
with open('all.json', 'w') as outfile:
    json.dump(combine, outfile, indent=2, ensure_ascii=True)

In [133]:
# Read file JSON
with open('all.json', 'r') as file:
    data = json.load(file)

# To DataFrame
df = pd.DataFrame(data)

# Show DataFrame
df


title              rating  \
0                     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   
1                      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars   
2                  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   
3                          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   
4                   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   
..                                                  ...                 ...   
730                   Reebok Classic Nylon (Little Kid)  4.6 out of 5 stars   
731           Reebok Women's Club C Grounds UK Sneakers  4.4 out of 5 stars   
732             Reebok Men's Club Memt Gum Classic Shoe  4.5 out of 5 stars   
733       Reebok Women's Club C Double Revenge Sneakers  4.6 out of 5 stars   
734  Reebok Work Zig Pulse Work Sneaker CT Men's Oxford  3.8 out of 5 stars   

    price  \
0     48.   
1    179.   
2     54.   
3     56.   
4     66.   
..    ...   
730   41.   
731   47.   
732   29.   
733   65.   
734   49.   

                                                                              img_url  \
0       https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1    https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2       https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3    https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4    https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   
..                                                                                ...   
730     https://m.media-amazon.com/images/I/71tgmXYfUxL._AC_SY395_SX395_QL70_ML2_.jpg   
731     https://m.media-amazon.com/images/I/51brEN-fGjL._AC_SY395_SX395_QL70_ML2_.jpg   
732  https://m.media-amazon.com/images/I/71TiRfHobkL._AC_SY395_SX395_QL70_FMwebp_.jpg   
733  https://m.media-amazon.com/images/I/7138dLkGonL._AC_SY395_SX395_QL70_FMwebp_.jpg   
734  https://m.media-amazon.com/images/I/61XU6NEQ2KL._AC_SY395_SX395_QL70_FMwebp_.jpg   

      Brand  \
0    Adidas   
1    Adidas   
2    Adidas   
3    Adidas   
4    Adidas   
..      ...   
730  Reebok   
731  Reebok   
732  Reebok   
733  Reebok   
734  Reebok   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Customer_say  \
0                                                                                                                                                                                                                          Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                                                                                                                               Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, wit

In [134]:
# Save to CSV as backup
df.to_csv('data_before_clean.csv', index=False)

In [135]:
# Data Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         735 non-null    object
 1   rating        735 non-null    object
 2   price         735 non-null    object
 3   img_url       735 non-null    object
 4   Brand         735 non-null    object
 5   Customer_say  589 non-null    object
 6   Review        735 non-null    object
dtypes: object(7)
memory usage: 40.3+ KB


In [136]:
# Change Column Name
df.rename(columns={
    'title': 'product',
    'rating': 'rating',
    'price': 'price',
    'img_url': 'img_url',
    'Brand': 'brand',
    'Customer_say': 'customer_say',
    'Review': 'review'
}, inplace=True)

df.head()


product              rating price  \
0     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   48.   
1      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars  179.   
2  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   54.   
3          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   56.   
4   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                        

# Create gender column and remove kid shoes

In [137]:
# Remove products for kids
df = df[~df['product'].str.contains('Kid', case=False, na=False)]

# Create gender column
df['gender'] = df['product'].apply(
    lambda x: (
        'Men' if 'Men' in x or "Men's" in x or 'Mens' in x
        else 'Women' if 'Women' in x or "Women's" in x or 'Womens' in x
        else 'Unisex'
    )
)

# Preview result
df.head()

product              rating price  \
0     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   48.   
1      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars  179.   
2  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   54.   
3          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   56.   
4   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                        

In [138]:
# Check brand
df.brand.unique()

array(['Adidas', 'New Balance', 'Nike', 'PUMA', 'Reebok'], dtype=object)

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681 entries, 0 to 734
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product       681 non-null    object
 1   rating        681 non-null    object
 2   price         681 non-null    object
 3   img_url       681 non-null    object
 4   brand         681 non-null    object
 5   customer_say  541 non-null    object
 6   review        681 non-null    object
 7   gender        681 non-null    object
dtypes: object(8)
memory usage: 47.9+ KB


# Data Cleaning

## Column product

drop missing value

In [140]:
# Change 'N/A' to NaN
df['product'] = df['product'].replace('N/A', np.nan)

In [141]:
# Drop Data null in products
df = df.dropna(subset=['product']).reset_index(drop=True)

memastikan ada brand di nama produk

In [142]:
# Filter products based on Brand

# Create regex pattern with keyword
pattern = r'^(Adidas|New Balance|Nike|PUMA|Reebok)'

# Filter products with keyword product (case-insensitive)
df = df[df['product'].str.contains(pattern, case=False, na=False, regex=True)]

# Reset index
df = df.reset_index(drop=True)


In [143]:
# Make sure brand is in product name
df = df[df.apply(lambda row: str(row['brand']).lower() in str(row['product']).lower(), axis=1)]

Memastikan data brand tercantum di kolom nama produk

In [144]:
df['product'].unique()

array(["adidas Men's Run Falcon 5 Sneaker",
       "adidas Men's Ultraboost 1.0 Shoe",
       "adidas Women's Run Falcon 5 Sneakers",
       "adidas Men's Kaptir 3.0 Shoe",
       "adidas Men's Ultradream DNA Sneaker",
       "adidas Men's Duramo Speed 2 Running Sneaker",
       "adidas Men's Racer TR23 Sneaker",
       "adidas Men's Lite Racer 3.0 Running Shoe",
       "adidas Men's Run Falcon 3.0 Shoe",
       "adidas Men's Kaptir 4.0 Sneaker",
       "adidas Men's X_PLR Path Sneaker",
       "adidas Men's Lightblaze Sneaker",
       "adidas Women's Response Super Sneaker",
       'adidas Womens X_PLR Path', "adidas Men's Run 72 Sneaker",
       "adidas Men's Run 84 Shoes Sneaker",
       "Adidas Men's Lightshift Sneaker",
       "adidas Men's Supernova Ease Running Sneaker",
       "adidas Men's Terrex Tracefinder Trail Running Sneaker",
       "adidas Women's Kaptir Flow Sneaker",
       "adidas Men's Ultrarun 5 Sneaker",
       "adidas Women's Run Falcon 5 Trail Sneaker",
       "

Menghapus setelah koma, (dalam kurung), dan kata kata seperti trail, running, and shoe yang berada pada nama produk.

In [145]:
# Combined cleaning pattern
pattern_words = r'\b(trail|running|shoe|shoes|sneaker|sneakers)\b'

df['product'] = (
    df['product']
    .str.replace(r'\([^)]*\)', '', regex=True)   # remove (...) and contents
    .str.replace(r',.*', '', regex=True)         # remove comma and everything after
    .str.replace(pattern_words, '', flags=re.IGNORECASE, regex=True)  # remove keywords
    # .str.replace(r'[^A-Za-z0-9\s]', '', regex=True)  # ❗ remove all symbols
    .str.replace(r'\s+', ' ', regex=True)        # normalize spaces
    .str.strip()
)



In [146]:
df['product'].unique()

array(["adidas Men's Run Falcon 5", "adidas Men's Ultraboost 1.0",
       "adidas Women's Run Falcon 5", "adidas Men's Kaptir 3.0",
       "adidas Men's Ultradream DNA", "adidas Men's Duramo Speed 2",
       "adidas Men's Racer TR23", "adidas Men's Lite Racer 3.0",
       "adidas Men's Run Falcon 3.0", "adidas Men's Kaptir 4.0",
       "adidas Men's X_PLR Path", "adidas Men's Lightblaze",
       "adidas Women's Response Super", 'adidas Womens X_PLR Path',
       "adidas Men's Run 72", "adidas Men's Run 84",
       "Adidas Men's Lightshift", "adidas Men's Supernova Ease",
       "adidas Men's Terrex Tracefinder", "adidas Women's Kaptir Flow",
       "adidas Men's Ultrarun 5", "adidas Women's Puremotion Adapt",
       "adidas Men's Supernova Prima", "adidas Men's Astrastar",
       "adidas Men's", "adidas Men's Ultraboost Light",
       "Adidas Men's Run Falcon 5", "adidas Men's Ultrabounce 5",
       "adidas Women's Cloudfoam Pure", 'Adidas Womens Swift Run 22',
       'adidas Womens Ru

## Column rating

Kolom rating perlu dilakukan striping setelah angka rating dan hasil diubah menjadi numerikal.

In [147]:
# Clean and Convert rating
df['rating'] = df['rating'].str.extract(r'(\d+\.?\d*)').astype(float)

In [148]:
df.head()

product  rating price  \
0    adidas Men's Run Falcon 5     4.4   48.   
1  adidas Men's Ultraboost 1.0     4.4  179.   
2  adidas Women's Run Falcon 5     4.5   54.   
3      adidas Men's Kaptir 3.0     4.5   56.   
4  adidas Men's Ultradream DNA     4.5   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                                                                                                                                             

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    object 
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


## Column price

kolom harga akan ditambahkan 1 dollar karena ada nilai sen yang tidak di scrap, tipenya juga akan diubah

In [150]:
# Change 'N/A' to NaN
df['price'] = df['price'].replace('N/A', np.nan)

In [151]:
# Convert to numeric (float), ignoring non-numeric characters
df['price'] = df['price'].str.replace('[^\d.]', '', regex=True).astype(float)

# Add 1
df['price'] = df['price'] + 1

In [152]:
df.head()

product  rating  price  \
0    adidas Men's Run Falcon 5     4.4   49.0   
1  adidas Men's Ultraboost 1.0     4.4  180.0   
2  adidas Women's Run Falcon 5     4.5   55.0   
3      adidas Men's Kaptir 3.0     4.5   57.0   
4  adidas Men's Ultradream DNA     4.5   67.0   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                                                                                                                                       

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


Kolom price sudah bersih dan berbentuk type numerical.

## Column img_url

In [154]:
# Change 'N/A' to NaN
df['img_url'] = df['img_url'].replace('N/A', np.nan)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


# Handling Missing Value

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


handling missing value customer_say

In [157]:
df['customer_say'] = df['customer_say'].fillna("No summary available.")

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  655 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


# Handling duplicate

cek duplikasi nama produk, menggabungkan review dan summary untuk produk yang sama

In [159]:
# 1) Convert numeric columns
for col in ['price', 'rating']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2) Safe join for text columns
def safe_join(values):
    result = []
    for v in values:
        if isinstance(v, list):
            result.extend(map(str, v))
        elif pd.notna(v) and str(v).strip() != '':
            result.append(str(v))
    return ' '.join(result)

# 3) Helper for taking single unique value (case-insensitive)
def unique_one(values):
    uniques = pd.Series(values).dropna().unique()
    if len(uniques) == 0:
        return None
    # Normalize case so "adidas" and "Adidas" count as same
    uniques = list({str(u).strip().title() for u in uniques})
    return uniques[0]

# 4) Aggregation rules
agg_dict = {c: safe_join for c in df.columns if c not in ['product', 'price', 'rating', 'brand', 'gender']}
agg_dict.update({
    'price': 'mean',
    'rating': 'mean',
    'brand': unique_one,
    'gender': unique_one
})

# 5) Group & aggregate
df_merged = (
    df.groupby('product', dropna=False)
      .agg(agg_dict)
      .reset_index()
)

# 6) Optional rounding
df_merged['price'] = df_merged['price'].round(2)
df_merged['rating'] = df_merged['rating'].round(2)


In [160]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       609 non-null    object 
 1   img_url       609 non-null    object 
 2   customer_say  609 non-null    object 
 3   review        609 non-null    object 
 4   price         609 non-null    float64
 5   rating        609 non-null    float64
 6   brand         609 non-null    object 
 7   gender        609 non-null    object 
dtypes: float64(2), object(6)
memory usage: 38.2+ KB


In [161]:
dupes = df_merged[df_merged['product'].duplicated(keep=False)]
print(dupes['product'].unique())

[]


In [162]:
# Save data_product
df_merged.to_csv('data_clean_raw_reviews.csv', index=False)